#  Comparativo de solver basado en multiplicadores de Lagrange y método de Newton

Para comparar el desempeño de ambos solvers, se decidió variar el valor del rendimiento $r$ en un rango de [WIP: espeficificar rango]. Al respecto, los resultados obtenidos se resumen en la siguiente tabla:

[WIP: añadir tabla que compare la norma de la diferencia y el valor absoluto entre ambos]

De lo anterior, se desprenden los siguientes hallazgos:

    [WIP: hallazgo 1]
    [WIP: hallazgo 2]
    [WIP: hallazgo 3]



## Librerías

In [0]:
import numpy as np
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [33]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [34]:
datos.head()

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,BABA,BHP.AX,BRK-A,CCI,COP,CVX,DLR,FB,GOOG,HD,HON,JNJ,JPM,KO,LIN,LMT,MA,MC.PA,MRK,MSFT,NSRGY,PEP,PFE,PG,PLD,RDSA.AS,RELIANCE.NS,RHHBY,RIO.L,RTX,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.50,7507.0,109.330002,89.786400,99.669998,308.519989,103.599998,27.603399,223600.0,79.510002,68.919998,112.580002,66.410004,78.449997,523.373108,103.430000,95.556229,104.519997,62.490002,42.139999,129.949997,193.309998,85.680000,130.850006,57.189999,46.759998,72.650002,94.440002,31.33,90.440002,43.430000,27.750000,442.774994,33.910000,2970.0,72.397736,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,101.000000,27.547300,220980.0,79.000000,65.639999,108.080002,67.690002,77.190002,512.463013,101.260002,93.735291,103.790001,60.549999,42.139999,126.519997,189.289993,83.269997,127.050003,58.040001,46.330002,70.959999,93.730003,31.16,90.010002,43.400002,26.615000,437.924988,34.029999,2883.5,71.189430,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,103.320000,26.267099,220450.0,78.849998,62.930000,108.029999,67.480003,76.150002,500.585632,100.949997,93.516014,103.279999,58.980000,42.459999,124.900002,188.399994,83.089996,125.599998,60.320000,45.650002,70.610001,93.019997,31.42,89.599998,43.549999,26.514999,418.049988,33.900002,2944.5,70.182503,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,102.129997,26.267099,223480.0,80.500000,63.349998,107.940002,68.019997,76.150002,499.727997,104.410004,94.192909,105.559998,59.070000,42.990002,126.300003,190.830002,84.220001,125.699997,61.610001,46.230000,70.750000,95.739998,31.85,90.070000,44.209999,26.870001,427.149994,33.990002,2962.5,70.943993,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,105.029999,26.519400,226680.0,81.769997,64.930000,110.410004,68.910004,78.180000,501.303680,106.720001,95.908974,106.389999,60.389999,43.509998,128.380005,195.130005,85.529999,129.649994,62.849998,47.590000,71.459999,97.480003,32.50,91.099998,44.220001,27.495001,421.024994,34.279999,3027.5,72.152298,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


In [0]:
mu = aux.calcular_rendimiento(datos)

In [0]:
S = aux.calcular_varianza(datos)

In [37]:
max(mu)

array(0.40221088)

In [0]:
rango =np.arange(0.4,1.1,0.1)

In [39]:
rango

array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [0]:
res_sol1 = [mkv.markowitz(r,mu,S) for r in rango]

In [0]:
fo = lambda w: w@S@w
#w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#b = cp.array([r,1])
M = cp.ones((2,mu.shape[0]))
tol=1e-8
tol_backtracking=1e-14
#p_ast=fo(w_ast)

In [46]:
res_sol2 = [opt.Newtons_method_feasible_init_point(fo,
                                                   A,
                                                   utils.feasible_markowitz(r,mu),
                                                   tol,
                                                   tol_backtracking,
                                                   mkv.markowitz(r,mu,S),
                                                   fo(mkv.markowitz(r,mu,S)),
                                                   maxiter=50)[0] for r in rango]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0057	0.0316	4.2131	168.5924	---		3691.1513
1	0.0057	0.0001	2.8707	0.689	1	3691.1513
2	0.0057	0.0	0.4834	0.0136	1	3691.1513
3	0.0057	0.0	0.0114	0.0	1	3691.1513
Error of x with respect to x_ast: 0.011407118501654568
Approximate solution: [ 1.58013068e-01 -2.06643048e-02  1.56888574e-01 -9.17697667e-02
  3.19319096e-02  5.14213637e-02  8.71116856e-02  1.31839974e-02
  3.69333218e-02  2.84612054e-01 -5.74568268e-03  2.14419490e-03
  2.02941587e-01  9.46233639e-02  2.25646324e-02  1.50063110e-02
  7.50109583e-03  2.99242688e-02  5.77178989e-02  1.97410870e-01
  1.18983078e-01  1.27790993e-01  1.40561445e-01  1.45237667e-02
  8.82707482e-02  1.49986139e-01  1.68304359e-01  7.64805620e-02
  8.05046704e-02  8.19865894e-02  1.92078725e-01 -2.39442398e-02
  2.68664187e-02  7.70619164e-02  2.15330621e-02 -1.07818459e-01
  1.63323248e-01  1.77447913e-02  6.13207967e-02 -1.07509909e-01
 -5.01056640e-02  1.38106609e-01  1.0314

In [48]:
res_sol1

[array([ 1.58055687e-01, -2.07133529e-02,  1.57019627e-01, -9.18345280e-02,
         3.19629621e-02,  5.13303564e-02,  8.66430485e-02,  1.31083846e-02,
         3.68038685e-02,  2.84543378e-01, -5.89917855e-03,  2.20489926e-03,
         2.02803941e-01,  9.44892356e-02,  2.24523214e-02,  1.48948486e-02,
         7.58279459e-03,  2.96976596e-02,  5.74939360e-02,  1.97095613e-01,
         1.19142470e-01,  1.27467863e-01,  1.39911588e-01,  1.41390597e-02,
         8.80176044e-02,  1.49721131e-01,  1.68100114e-01,  7.64488071e-02,
         8.04206308e-02,  8.15996592e-02,  1.92053687e-01, -2.42807056e-02,
         2.66693146e-02,  7.68679771e-02,  2.15276412e-02, -1.07682220e-01,
         1.63429206e-01,  1.78417675e-02,  6.13589655e-02, -1.07613225e-01,
        -5.02574035e-02,  1.37870955e-01,  1.03079127e-01, -4.23433640e-02,
         1.06280630e-02, -2.32468349e+00,  1.92867098e-01,  2.62640967e-01,
         1.19875252e-01, -1.04554037e-01]),
 array([ 0.17591158, -0.02784513,  0.2036972

In [47]:
res_sol2

[array([ 1.58013068e-01, -2.06643048e-02,  1.56888574e-01, -9.17697667e-02,
         3.19319096e-02,  5.14213637e-02,  8.71116856e-02,  1.31839974e-02,
         3.69333218e-02,  2.84612054e-01, -5.74568268e-03,  2.14419490e-03,
         2.02941587e-01,  9.46233639e-02,  2.25646324e-02,  1.50063110e-02,
         7.50109583e-03,  2.99242688e-02,  5.77178989e-02,  1.97410870e-01,
         1.18983078e-01,  1.27790993e-01,  1.40561445e-01,  1.45237667e-02,
         8.82707482e-02,  1.49986139e-01,  1.68304359e-01,  7.64805620e-02,
         8.05046704e-02,  8.19865894e-02,  1.92078725e-01, -2.39442398e-02,
         2.68664187e-02,  7.70619164e-02,  2.15330621e-02, -1.07818459e-01,
         1.63323248e-01,  1.77447913e-02,  6.13207967e-02, -1.07509909e-01,
        -5.01056640e-02,  1.38106609e-01,  1.03146825e-01, -4.21575744e-02,
         1.05201035e-02, -2.34740451e+00,  2.09224218e-01,  2.62758495e-01,
         1.20075130e-01, -1.03962776e-01]),
 array([ 0.17582158, -0.02786526,  0.2036075

In [53]:
for i in range(7):
  print(np.linalg.norm(res_sol1[i]-res_sol2[i]))

0.02803902021058542
0.014771181148932552
0.09540182407920716
77.50931169410492
33.90617581958757
0.031520561598820676
0.043917100795634864


In [69]:
for i in range(7):
  print("------------{}-------------".format(i))
  print(sum(res_sol1[i]*mu))
  print(sum(res_sol1[i]*mu))

------------0-------------
0.4000000000000007
0.4000000000000007
------------1-------------
0.5000000000000004
0.5000000000000004
------------2-------------
0.6000000000000003
0.6000000000000003
------------3-------------
0.7000000000000005
0.7000000000000005
------------4-------------
0.8000000000000006
0.8000000000000006
------------5-------------
0.9000000000000008
0.9000000000000008
------------6-------------
1.0000000000000007
1.0000000000000007


In [70]:
for i in range(7):
  print("------------{}-------------".format(i))
  print(sum(res_sol1[i]))
  print(sum(res_sol1[i]))

------------0-------------
1.0000000000000004
1.0000000000000004
------------1-------------
1.0000000000000007
1.0000000000000007
------------2-------------
0.9999999999999998
0.9999999999999998
------------3-------------
1.0000000000000007
1.0000000000000007
------------4-------------
1.000000000000001
1.000000000000001
------------5-------------
1.0
1.0
------------6-------------
0.9999999999999993
0.9999999999999993
